<a href="https://colab.research.google.com/github/vcn2fj/ds2002_hw2/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
import sqlite3

# Create or connect to an SQLite database
connection = sqlite3.connect('student_grades.db')

# Create a cursor object (controller that connects to db)
cursor = connection.cursor()

Database created and connected!


Cursor created!


In [ ]:
#2
# Create the students table
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT,
    last_name TEXT
)
''')

# Create the grades table
cursor.execute('''
CREATE TABLE IF NOT EXISTS grades (
    grade_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER REFERENCES students(student_id),
    subject TEXT,
    grade INTEGER
)
''')

In [ ]:
cursor.execute("DELETE FROM students")
cursor.execute("DELETE FROM grades")
connection.commit()

In [ ]:
#3
# Sample data to insert into the students table
students_data = [
    (1, 'Alice', 'Johnson'),
    (2, 'Bob', 'Smith'),
    (3, 'Carol', 'White'),
    (4, 'David', 'Brown'),
    (5, 'Eve', 'Davis')
]

# Insert data into the table
cursor.executemany('''
INSERT INTO students (student_id, first_name, last_name) VALUES (?, ?, ?)
''', students_data)

# Commit the transaction
connection.commit()

# Sample data to insert into the grades table
grades_data = [
    (1, 1, 'Math', 95),
    (2, 1, 'English', 88),
    (3, 1, 'History', 90),
    (4, 2, 'Math', 82),
    (5, 2, 'English', 82),
    (6, 2, 'History', 85),
    (7, 3, 'Math', 95),
    (8, 3, 'English', 88),
    (9, 3, 'History', 90),
    (10, 4, 'Math', 82),
    (11, 4, 'English', 79),
    (12, 4, 'History', 80),
     (13, 5, 'Math', 95),
    (14, 5, 'English', 99),
    (15, 5, 'History', 94),
]

# Insert data into the table
cursor.executemany('''
INSERT INTO grades (grade_id, student_id, subject, grade) VALUES (?, ?, ?, ?)
''', grades_data)

# Commit the transaction
connection.commit()

In [ ]:
#4
# # Fetch all data from the sales table
# cursor.execute("SELECT * FROM students")
# rows = cursor.fetchall()

# # Print the results
# print("All sales data:")
# for row in rows:
#     print(row)

# Perform a JOIN query between students and grades
cursor.execute('''
SELECT students.first_name, students.last_name, grades.grade
FROM students
JOIN grades ON students.student_id = grades.student_id
''')

# Fetch and display the results
joined_data = cursor.fetchall()

print("Joined student names and their grades:")
for row in joined_data:
    print(row)

Joined student names and their grades:
('Alice', 'Johnson', 95)
('Alice', 'Johnson', 88)
('Alice', 'Johnson', 90)
('Bob', 'Smith', 82)
('Bob', 'Smith', 82)
('Bob', 'Smith', 85)
('Carol', 'White', 95)
('Carol', 'White', 88)
('Carol', 'White', 90)
('David', 'Brown', 82)
('David', 'Brown', 79)
('David', 'Brown', 80)
('Eve', 'Davis', 95)
('Eve', 'Davis', 99)
('Eve', 'Davis', 94)


In [ ]:
# Query for the average revenue per flavor
cursor.execute('''
SELECT AVG(grades) FROM sales
''')
average_revenue = cursor.fetchone()[0]

# Display the result
print(f"Average revenue per flavor: ${average_revenue}")